In [5]:
from pandas import set_option as pd_set_option
pd_set_option('display.max_rows', 500)
pd_set_option('display.max_columns', 500)
from pandas import concat as pd_concat
from pandas import merge as pd_merge
from pandas import MultiIndex as pd_MultiIndex
from reports.contracts.contract import  get_group_fields
from pandas import DataFrame as pd_DataFrame
from json import load as js_load
from common import full_path

def get_contract_report():
    with open(full_path("reports/contracts/files/full.json"), "r", encoding='utf-8') as js_file:
        data_load = js_load(js_file)

    df_period = pd_DataFrame(data_load["dannieperiod"])
    df_rev = pd_DataFrame(data_load["dohdogovori"])
    df_exp = pd_DataFrame(data_load["rashdogovori"])

    """union and merge table"""
    df_result = pd_concat([df_rev, df_exp], ignore_index=True, sort=False)
    # df_result = df_result.loc[df_result['doh_dogovor'] == '"Договор 00000006249 от 09.08.2019 12:00:01"']
    df_result = pd_merge(df_result, df_period, on=["dogovorssilka", "filial"], how="left")
    df_result = df_result.fillna(
        {"period": "2019-01-01T00:00:00", "oplatazaperiodplan": 0, "oplatazaperiodfakt": 0, "summazaperiodplanitogo": 0,
         "viruchkazaperiodfakt": 0}).fillna('')
    """building pivot table"""
    df_result = df_result.pivot_table(index=get_group_fields(), columns="period",
                                      values=["oplatazaperiodplan", "oplatazaperiodfakt", "summazaperiodplanitogo",
                                              "viruchkazaperiodfakt"], fill_value='',
                                      margins=True,
                                      aggfunc=sum)
    """removing the last row with totals"""
    df_result = df_result[:-1]
    """to swipe columns"""
    # result = result.swaplevel(0, 1, axis=1).sort_index(axis=1)
    """reseting and building a new hierarchy indexes"""
    df_result = df_result.reset_index()
    df_result = df_result.set_index(['doh_dogovor', 'rash_dogovor']).sort_index()

    """format values(need to rebuild later)"""
    df_result['doh_data'] = df_result['doh_data'].apply(lambda x: "{:.10}".format(x))
    df_result['doh_dataokonchaniya'] = df_result['doh_dataokonchaniya'].apply(lambda x: "{:.10}".format(x))
    df_result['rash_data'] = df_result['rash_data'].apply(lambda x: "{:.10}".format(x))
    
    df_result = df_result.replace(0, '')
    
    return df_result

get_contract_report()

filial  \
period                                                                                                         
doh_dogovor                                rash_dogovor                                                        
Договор 00000000052 от 25.01.2012 13:03:37                                             Центральный офис (ЦО)   
                                           Договор 00000000172 от 13.02.2012 10:48:20  Центральный офис (ЦО)   
                                           Договор 00000000191 от 15.10.2012 9:48:27   Центральный офис (ЦО)   
                                           Договор 00000001258 от 27.03.2013 12:00:00  Центральный офис (ЦО)   
                                           Договор 00000002000 от 01.08.2014 12:00:00  Центральный офис (ЦО)   
...                                                                                                      ...   
Договор 00000006349 от 09.09.2019 0:00:00                                              Центральный офис (ЦО)   
                                           Договор 00000006351 от 20.09.2019 12:00:00  Центральный офис (ЦО)   
Договор 00000006355 от 29.08.2019 0:00:00                                              Центральный офис (ЦО)   
Договор 00000006364 от 27.11.2019 0:00:00                                              Центральный офис (ЦО)   
Договор 00000006366 от 27.11.2019 0:00:00                                              Центральный офис (ЦО)   

                                                                                                                    dogovorssilka  \
period                                                                                                                              
doh_dogovor                                rash_dogovor                                                                             
Договор 00000000052 от 25.01.2012 13:03:37                                             Договор 00000000052 от 25.01.2012 13:03:37   
                                           Договор 00000000172 от 13.02.2012 10:48:20  Договор 00000000172 от 13.02.2012 10:48:20   
                                           Договор 00000000191 от 15.10.2012 9:48:27    Договор 00000000191 от 15.10.2012 9:48:27   
                                           Договор 00000001258 от 27.03.2013 12:00:00  Договор 00000001258 от 27.03.2013 12:00:00   
                                           Договор 00000002000 от 01.08.2014 12:00:00  Договор 00000002000 от 01.08.2014 12:00:00   
...                                                                                                                           ...   
Договор 00000006349 от 09.09.2019 0:00:00                                               Договор 00000006349 от 09.09.2019 0:00:00   
                                           Договор 00000006351 от 20.09.2019 12:00:00  Договор 00000006351 от 20.09.2019 12:00:00   
Договор 00000006355 от 29.08.2019 0:00:00                                               Договор 00000006355 от 29.08.2019 0:00:00   
Договор 00000006364 от 27.11.2019 0:00:00                                               Договор 00000006364 от 27.11.2019 0:00:00   
Договор 00000006366 от 27.11.2019 0:00:00                                               Договор 00000006366 от 27.11.2019 0:00:00   

                                                                                                                doh_kontragent  \
period                                                                                                                           
doh_dogovor                                rash_dogovor                                                                          
Договор 00000000052 от 25.01.2012 13:03:37                                             Объединенная энергетическая компания АО   
                                           Договор 00000000172 от 13.02.2012 10:48:20                                            
                    